In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import string
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

### Preprocessing the data

Emg-window size is 150

Nemg-window size is 55

In [ ]:
dir="../input/imu-dataset-original/IMU dataset"

#### Data Preprocesing::

def funct_preprocessing(l,f,fn):
    
    emg=pd.DataFrame(columns=['timestamp', 'emg1', 'emg2', 'emg3', 'emg4', 'emg5', 'emg6', 'emg7',
       'emg8'])
    acc=pd.DataFrame()
    gyro=pd.DataFrame()
    
    for i in sorted(os.listdir(os.path.join(dir,l))):
        
        if "emg" in i:
            emg=pd.concat([emg,pd.read_csv(os.path.join(dir,l,i))],axis=0)
            
        elif "acc" in i or "gyro" in i:  
        #### accelerometer,gyro,orientation have same feature names. Hence changing the column names
            if "acc" in i:
                temp=pd.read_csv(os.path.join(dir,l,i))
                acc=pd.concat([acc,temp])

            elif "gyro" in i:
                temp=pd.read_csv(os.path.join(dir,l,i))
                gyro=pd.concat([gyro,temp])


    acc.columns=["timestamp","x_acc","y_acc","z_acc"]
    acc.drop("timestamp",axis=1,inplace=True)
    gyro.columns=["timestamp","x_gyro","y_gyro","z_gyro"]
    
    nemg=pd.concat([acc,gyro],axis=1)
    
    ### Dropping the timestamp column:
    emg.drop("timestamp",axis=1,inplace=True)
    nemg.drop("timestamp",axis=1,inplace=True)
    
    #### preprocess emg and nemg further
    ### Emg-window size is 150
    ### Nemg-window size is 55

    ne=emg.shape[0]//150
    nn=nemg.shape[0]//55
    
    emg=emg.iloc[0:(ne)*150,:]
    nemg=nemg.iloc[0:(nn)*55,:]

    emg_new=pd.DataFrame(columns=[i for i in range(150)])
    ### for emg:
    for i in range(ne):   ### points to row number
        for j in range(emg.shape[1]):   ### columns number
            k=pd.DataFrame((emg.iloc[i*150:(i+1)*150,j].values).reshape(1,-1))
            k1=k.to_dict('records')
            emg_new=emg_new.append(k1)
    emg_new["label"]=l

    nemg_new=pd.DataFrame(columns=[i for i in range(55)])
    ### for emg:
    for i in range(nn):   ### points to row number
        for j in range(nemg.shape[1]):   ### columns number
            k=pd.DataFrame((nemg.iloc[i*55:(i+1)*55,j].values).reshape(1,-1))
            k1=k.to_dict('records')
            nemg_new=nemg_new.append(k1)
    nemg_new["label"]=l    
    
    f=pd.concat([f,emg_new],axis=0)
    fn=pd.concat([fn,nemg_new],axis=0)
    
    return f,fn

In [ ]:
%%time
### total time for 5 classes is 14.5 mins:
f=pd.DataFrame(columns=[i for i in range(150)]+["label"])
fn=pd.DataFrame(columns=[i for i in range(55)]+["label"])
for i in list(string.ascii_uppercase)[:5]:
    f,fn=funct_preprocessing(i,f,fn)

#### Label Encoding the target variable:
le=LabelEncoder()
f["label"]=le.fit_transform(f["label"])
fn["label"]=le.fit_transform(fn["label"])

%%time
f.head()

### Exporting the dataframe into csv::
f.to_csv("sample_5_classes_emg.csv",index=False)
fn.to_csv("sample_5_classes_nemg.csv",index=False)

### Trying the Time Series Transformers Model::

In [ ]:
df=pd.read_csv("../input/sample-5-classes-data/sample_5_classes_emg.csv")
X=df.drop("label",axis=1).values
y=df["label"].values

### Splitting the data
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

### Reshaping it into a 3-D dimension
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

In [ ]:
### Sample Data (Default Example in Documentation)

# def readucr(filename):
#     data = np.loadtxt(filename, delimiter="\t")
#     y = data[:, 0]
#     x = data[:, 1:]
#     return x, y.astype(int)


# root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

# X_train, Y_train = readucr(root_url + "FordA_TRAIN.tsv")
# X_test, Y_test = readucr(root_url + "FordA_TEST.tsv")

# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

Our model processes a tensor of shape **(batch size, sequence length, features)**, where sequence length is the number 

of time steps and features is each input timeseries.

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res



def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

### Training and Validating:::    
%%time

input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.0,
    dropout=0.0,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)